In [ ]:
# !pip install --upgrade pixiedust
import pixiedust
pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
### TODO Please provide your Cloudant credentials in this cell
def readDataFrameFromCloudant(database):

  cloudantdata = spark.read.format("org.apache.bahir.cloudant")\
    .option("cloudant.host",'XXXX-bluemix.cloudant.com')\
    .option("cloudant.username", 'XXXX-bluemix')\
    .option("cloudant.password",'XXXX')\
    .load(database)
    
  return cloudantdata

In [ ]:
df=readDataFrameFromCloudant('training')

In [ ]:
# Enable SQL on the data frame
df.createOrReplaceTempView('df')

In [ ]:
from pyspark.sql.functions import translate, col

df_cleaned = df \
    .withColumn("temp", df.temp.cast('double')) \
    .withColumn("humidity", df.humidity.cast('double')) \

df_cleaned.createOrReplaceTempView('df_cleaned')
df_cleaned.select('temp', 'humidity').distinct().show()

In [ ]:
df_class_0 = spark.sql('select time, temp, humidity, class from df_cleaned where class = 0')
df_class_1 = spark.sql('select time, temp, humidity, class from df_cleaned where class = 1')
df_class_0.createOrReplaceTempView('df_class_0')
df_class_1.createOrReplaceTempView('df_class_1')

In [ ]:
df_class_1.select('temp', 'humidity').distinct().show()

In [ ]:
df_class_0.select('temp', 'humidity').distinct().show()

In [ ]:
df_cleaned.printSchema()

In [ ]:
spark.sql('select class, count(class) from df_cleaned group by class').show()

In [ ]:
display(df_class_0)

In [ ]:
# Imports for modelling
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# create binary classifier model
vectorAssembler = VectorAssembler(inputCols=["humidity","temp"],
                                  outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df_cleaned)
result = model.transform(df_cleaned)

In [ ]:
model.stages[1].coefficients

In [ ]:
model.stages[1].intercept

In [ ]:
#evaluate classification accuracy (1.0 = 100% accurate)
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")    
binEval.evaluate(result)

In [ ]:
# test the model
# re-read data from cloudant
new_df = readDataFrameFromCloudant('training')
new_df_cleaned = new_df \
    .withColumn("temp", new_df.temp.cast('double')) \
    .withColumn("humidity", new_df.humidity.cast('double')) \

new_df_cleaned.createOrReplaceTempView('df_cleaned')

result = model.transform(new_df_cleaned)
result.createOrReplaceTempView('result')
spark.sql("select humidity, temp, class, prediction from result").show(50)